In [1]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv('BTCUSD_1h.csv') # df = pd.read_csv('BTCUSD_training_data_1m.csv')
df = df.drop(['High', 'Low', 'Volume'], axis=1) # cleaning the df out of unnecessary data
df = df.set_index('Date') # Setting the index to the Date column
df['Value'] = df.mean(axis=1) # reducing the data even further by calculating the mean of Open and Close
df = df.drop(['Open', 'Close'], axis=1) 

In [3]:
def DCA(df, amount, start, end, period=1):

    hodl = 0
    invested = 0

    data = df['Value'].loc[start:end:period]
 
    for item in data:
        purchase = amount / item # iterating over the data set by calling the index in the df['Open'] series
        hodl+=purchase
        invested+=amount

    current_val = df['Value'].iat[-1] * hodl # using iat() instead of iloc() because we only need a single value and iat() is faster
    profit = current_val-invested
    roi = (profit/invested) * 100 # return of investment - investment / investment * 100

    return {'profit': profit, 'roi': roi, 'hodl': hodl, 'invested': invested, 'current_val': current_val}

In [4]:
start = '2020-01-01 00:00:00'
end = '2021-01-01 00:00:00'
amount = 10
period = 168 # hours 

In [11]:
result = DCA(df, amount, start, end, period)
invested = result['invested']
current_val = result['current_val']
hodl = result['hodl']
profit = result['profit']
roi = result['roi']

print(f'You have invested {invested} USD, and the current value of your holdings is {current_val:.2f} USD.')
print(f'You currently own {hodl:.5f} BTC and your profit is {profit:.2f} USD. Your ROI is {roi:.2f}%.')

You have invested 530 USD, and the current value of your holdings is 2107.87 USD.
You currently own 0.05313 BTC and your profit is 1577.87 USD. Your ROI is 297.71%.


In [12]:
def DCA_maximizer(budget, start, end, stepsize=24):
    format = "%Y-%m-%d %H:%M:%S" # specifying the format for the datetime.strptime() function
    dt_start = datetime.datetime.strptime(start, format)
    dt_end = datetime.datetime.strptime(end, format)
    hour_diff = (dt_end-dt_start).total_seconds() / 3600 # hour difference, because we have a data set with 1h data points

    dca_dict = dict()

    for period in range(24, 168+1, stepsize): # start at 1 day, end at 1 week, stepsize in hours
        n_investments = hour_diff / period 
        amount = budget / n_investments
        dca_dict[period] = DCA(df, amount, start, end, period)['profit']

    maximum = max(dca_dict.values())
    max_keys = [k for k, v in dca_dict.items() if v == maximum]

    print(f'It is the best to invest every {max_keys[0]} hours for the profit of {maximum:.2f} USD.')



In [13]:
DCA_maximizer(530, start, end)

It is the best to invest every 144 hours for the profit of 1609.92 USD.
